In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

import numpy as np
import arrayfire as af
from matplotlib import pyplot as plt

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import wave_equation_2d as w2d
from dg_maxwell import isoparam
from dg_maxwell import lagrange
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

In [3]:
params.mesh_file = '../read_and_plot_mesh/mesh/particle_in_rectangle.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 988 nodes.
gmshTranslator: Mesh has 374 elements.
gmshTranslator: Processed 1366 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


# Vectorise the Lax-Friedrichs flux

In [8]:
element_lf_flux = af.np_to_af_array(np.zeros([advec_var.elements.shape[0],
                                              4, params.N_LGL]))

for element_0_tag in np.arange(advec_var.u_e_ij.shape[1]):
    for element_0_edge_id, element_1_tag in enumerate(
        advec_var.interelement_relations[element_0_tag]):
        if element_1_tag != -1:
            element_1_tag = advec_var.interelement_relations[element_0_tag,
                                                             element_0_edge_id]

            element_1_edge_id = np.where(
                advec_var.interelement_relations[element_1_tag] \
                == element_0_tag)[0][0]

            u_element_0 = advec_var.u_e_ij[:, element_0_tag]
            u_element_1 = advec_var.u_e_ij[:, element_1_tag]

            u_element_0_at_edge = af.flat(a2d_arbit_mesh.u_at_edge(u_element_0,
                                                                   element_0_edge_id))
            u_element_1_at_edge = af.flat(a2d_arbit_mesh.u_at_edge(u_element_1,
                                                                   element_1_edge_id))

            element_lf_flux[element_0_tag,
                            element_0_edge_id] = a2d_arbit_mesh.lax_friedrichs_flux(
                                u_element_0_at_edge,
                                u_element_1_at_edge)

        if element_1_tag == -1:
            print('Element {} Tag at edge {} = -1'.format(element_0_tag,
                                                          element_0_edge_id))
            u_element_0 = advec_var.u_e_ij[:, element_0_tag]

            u_element_0_at_edge = af.flat(a2d_arbit_mesh.u_at_edge(u_element_0,
                                                    element_0_edge_id))
            u_element_1_at_edge = af.np_to_af_array(np.zeros([params.N_LGL]))

            element_lf_flux[element_0_tag,
                            element_0_edge_id] = a2d_arbit_mesh.lax_friedrichs_flux(
                                u_element_0_at_edge,
                                u_element_1_at_edge)

print('Done')


Element 59 Tag at edge 2 = -1
Element 114 Tag at edge 1 = -1
Element 165 Tag at edge 0 = -1
Element 229 Tag at edge 1 = -1
Done
